# 필요한 라이브러리 import

In [18]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [19]:
with open('./models/forward_ss.pickle','rb') as f:
    forward_ss = pickle.load(f)
    
with open('./models/forward_qt.pickle','rb') as f:
    forward_qt = pickle.load(f)

with open('./models/forward_mm.pickle','rb') as f:
    forward_mm = pickle.load(f)

forward_model = load_model('./models/forward_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [20]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [21]:
forward_df = premier_df[premier_df['포지션']=='공격수']

In [22]:
use_col = ['골','도움','크로스 성공률','파울 획득','반칙 수/90', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90', '인터셉트/90',
       '달린 거리/90분', '공중볼시도/90', '태클 성공률', '경기당 패스 시도/90', '경기당 패스 성공/90',
       '키패스/90', '태클 성공/90', '드리블 성공/90', '헤더 성공/90', '크로스 시도/90','평균획득승점/경기']

In [23]:
outlier_features = ['기회 창출/90', '반칙 수/90', '크로스 성공률', '헤더 성공/90', '파울 획득', '태클 성공/90']
nonoutlier_features = ['경기당 슈팅/90','키패스/90','드리블 성공/90','인터셉트/90','태클 성공률','달린 거리/90분',
                       '공중볼시도/90','경기당 패스 성공/90','경기당 유효 슈팅/90','크로스 시도/90','경기당 패스 시도/90']
order_features = ['골','도움']

In [24]:
forward_df.loc[:,outlier_features] = forward_qt.transform(forward_df.loc[:,outlier_features])
forward_df.loc[:,nonoutlier_features] = forward_ss.transform(forward_df.loc[:,nonoutlier_features])
forward_df.loc[:,order_features] = forward_mm.transform(forward_df.loc[:,order_features])

In [25]:
forward_df = forward_df.reset_index(drop=True)

In [26]:
good_features = ['골','도움','경기당 유효 슈팅/90','공중볼시도/90','경기당 패스 시도/90','기회 창출/90','드리블 성공/90']

In [27]:
forward_df['예측PPG']=forward_model.predict(tf.constant(forward_df.loc[:,good_features].values))
forward_df

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,0.194444,0.105263,0,7,11,...,1.907102,-0.565198,-0.613374,-1.104166,-0.501298,-0.656151,-1.521714,-0.998534,0.94,1.173140
1,Milot Rashica,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,0.111111,0.315789,0,8,9,...,0.615068,0.273872,0.202855,0.819425,0.242903,0.738278,-1.548865,2.187099,1.03,1.136479
2,Christos Tzolis,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,0.027778,0.000000,0,7,7,...,0.061339,0.055398,-0.096429,0.694516,0.979331,-0.203259,1.044273,0.329184,1.04,1.032862
3,Callum Wilson,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,0.472222,0.105263,0,20,5,...,-0.584678,-1.448597,-1.694878,-0.479623,-0.698526,-0.811087,0.005771,-0.757941,1.81,1.589676
4,Allan Saint-Maximin,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,0.111111,0.315789,0,22,6,...,0.522780,-0.790006,-0.810629,0.969315,1.054448,1.215006,-1.377524,0.436115,1.76,1.532176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Odsonne Edouard,2021-2022,공격수,팰리스,잉글랜드 프리미어 리그,0.583333,0.052632,0,15,11,...,-0.953831,-0.979984,-0.967073,-0.854349,-1.096804,-0.989860,0.134778,-0.766852,1.27,1.516620
63,Wilfried Zaha,2021-2022,공격수,팰리스,잉글랜드 프리미어 리그,0.250000,0.315789,0,14,9,...,0.338204,1.340917,1.376184,0.519645,0.211243,1.524879,-1.639431,0.231165,1.34,1.508812
64,Harry Kane,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,0.694444,0.631579,0,25,16,...,-1.507560,-0.865997,-0.885450,0.119937,-0.870846,-1.001778,0.456302,-0.833683,1.82,1.889886
65,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,0.583333,0.315789,0,22,17,...,-0.215526,0.964127,1.093905,-0.329733,-0.037988,1.048151,-0.946265,0.378194,1.66,2.016886


In [28]:
forward_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
60,Kai Havertz,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,0.611111,0.421053,0,34,11,...,0.338204,2.658099,2.546112,1.593858,0.145271,0.142368,-0.587360,-0.316853,2.05,2.125316
20,Raheem Sterling,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,0.277778,0.315789,0,28,10,...,0.430492,2.464955,2.478093,1.269095,1.028387,2.323398,-0.939671,0.944034,1.88,2.092985
18,Phil Foden,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,0.222222,0.263158,0,21,2,...,-0.215526,3.224868,3.267115,1.369022,0.954227,0.809787,-1.856781,0.779183,2.50,2.088428
12,Mohamed Salah,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,0.527778,0.421053,0,32,13,...,0.430492,0.976792,0.726602,1.493931,0.076032,0.809787,-0.926605,1.541062,1.98,2.070638
65,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,0.583333,0.315789,0,22,17,...,-0.215526,0.964127,1.093905,-0.329733,-0.037988,1.048151,-0.946265,0.378194,1.66,2.016886


In [29]:
forward_min = np.min(forward_df['예측PPG'].tolist())
forward_min

0.9501389265060425

In [30]:
forward_df['WAR'] = forward_df['예측PPG'].map(lambda x : x-forward_min+1)

In [31]:
forward_df.sort_values(by='WAR',ascending=False).head(60)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
60,Kai Havertz,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,0.611111,0.421053,0,34,11,...,2.658099,2.546112,1.593858,0.145271,0.142368,-0.587360,-0.316853,2.05,2.125316,2.175177
20,Raheem Sterling,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,0.277778,0.315789,0,28,10,...,2.464955,2.478093,1.269095,1.028387,2.323398,-0.939671,0.944034,1.88,2.092985,2.142846
18,Phil Foden,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,0.222222,0.263158,0,21,2,...,3.224868,3.267115,1.369022,0.954227,0.809787,-1.856781,0.779183,2.50,2.088428,2.138289
12,Mohamed Salah,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,0.527778,0.421053,0,32,13,...,0.976792,0.726602,1.493931,0.076032,0.809787,-0.926605,1.541062,1.98,2.070638,2.120499
65,Heung-Min Son,2021-2022,공격수,토트넘,잉글랜드 프리미어 리그,0.583333,0.315789,0,22,17,...,0.964127,1.093905,-0.329733,-0.037988,1.048151,-0.946265,0.378194,1.66,2.016886,2.066747
10,Diogo Jota,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,0.194444,0.157895,0,18,7,...,0.770982,0.917055,1.418986,1.423787,1.536797,0.323829,1.233637,1.74,1.973073,2.022934
9,Sadio Mané,2021-2022,공격수,리버풀,잉글랜드 프리미어 리그,0.361111,0.210526,0,25,11,...,1.926683,2.189012,0.369754,1.200579,1.608306,0.026939,1.033142,1.91,1.966897,2.016758
58,Romelu Lukaku,2021-2022,공격수,첼시,잉글랜드 프리미어 리그,1.000000,0.000000,0,36,10,...,-0.698183,-0.671190,-1.104166,-1.690622,-0.453541,0.338706,-0.909425,2.03,1.960497,2.010359
22,Cristiano Ronaldo,2021-2022,공격수,맨 레드,잉글랜드 프리미어 리그,0.583333,0.473684,0,22,13,...,-0.296062,-0.293684,0.844407,-0.037988,-0.012568,0.326799,-0.160913,1.80,1.954579,2.004440
19,Riyad Mahrez,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,0.194444,0.368421,0,20,3,...,1.432740,1.437401,2.318327,0.714776,1.250760,-1.923227,1.402943,2.42,1.894762,1.944623


In [38]:
forward_df.query(f'이름 in ["Phil Foden"]')[['이름','WAR']]

,이름,WAR
18,Phil Foden,2.138289


In [33]:
forward_df.loc[:,outlier_features] = forward_qt.inverse_transform(forward_df.loc[:,outlier_features])
forward_df.loc[:,nonoutlier_features] = forward_ss.inverse_transform(forward_df.loc[:,nonoutlier_features])
forward_df.loc[:,order_features] = forward_mm.inverse_transform(forward_df.loc[:,order_features])

In [39]:
forward_df[['이름','시즌','골', '도움', '경기당 유효 슈팅/90', '공중볼시도/90','경기당 패스 시도/90','기회 창출/90','드리블 성공/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,골,도움,경기당 유효 슈팅/90,공중볼시도/90,경기당 패스 시도/90,기회 창출/90,드리블 성공/90,WAR
60,Kai Havertz,2021-2022,22.0,8.0,1.44,6.66,40.82,0.31,1.65,2.175177
20,Raheem Sterling,2021-2022,10.0,6.0,1.24,6.03,40.21,0.30,3.48,2.142846
18,Phil Foden,2021-2022,8.0,5.0,1.26,4.57,42.61,0.37,2.21,2.138289
12,Mohamed Salah,2021-2022,19.0,8.0,1.71,6.18,35.51,0.46,2.21,2.120499
65,Heung-Min Son,2021-2022,21.0,6.0,1.48,8.10,35.47,0.30,2.41,2.066747
10,Diogo Jota,2021-2022,7.0,3.0,1.70,6.76,34.86,0.21,2.82,2.022934
9,Sadio Mané,2021-2022,13.0,4.0,1.25,8.31,38.51,0.38,2.88,2.016758
58,Romelu Lukaku,2021-2022,36.0,0.0,2.28,11.00,30.22,0.27,1.15,2.010359
22,Cristiano Ronaldo,2021-2022,21.0,9.0,1.95,8.86,31.49,0.43,1.52,2.004440
19,Riyad Mahrez,2021-2022,7.0,7.0,1.29,4.80,36.95,0.35,2.58,1.944623
